# Segmenting and Clustering Neighbourhoods in Toronto

A Coursera Data Science Capstone Assignment

In [91]:
#import necessary modules
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Web Scraping for Toronto Neighbourhood Data Set

The data will be scraped from wikipedia at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [89]:
#install needed packages
!pip install lxml html5lib beautifulsoup4 folium
#!pip install folium
#import folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import json # library to handle JSON files

from sklearn.cluster import KMeans

In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(URL)

In [4]:
print('There are {} tables on the page'.format(len(dfs))) #this show that there are 

There are 3 tables on the page


In [5]:
df = dfs[0] #Inspection shows that our table of interest is the first
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Renaming column Postal Code to PostalCode

In [6]:
df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df.drop(df[df.Borough=='Not assigned'].index,inplace=True)
df.index = range(len(df))
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.  These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [8]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [9]:
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x: df['Borough'] if x == 'Not Assigned' else x)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Assumptions: 
- The website is available
- The table of interest is available as the first on the page - index 0
- The Schema of the tables are assumed consistent as-is

In [10]:
df.shape

(103, 3)

## 2. Integrating Latitude and Longitude for each Neighbourhod

In [11]:
CLIENT_ID = 'O1VAWMONTH2GQ5MB1WYOSWR1IUCYU3C1ODPTYWIZQFNXWNTF' # your Foursquare ID
CLIENT_SECRET = 'LOX2OZZDHUQFVY1N2NEDLY0JS4QNIUWQYY0CPVYZJPFCWTKD' # your Foursquare Secret
ACCESS_TOKEN = 'HI5GYLJH05GATPAGPIQTKOSY4DYCV5CBX2G2DV2LWIP12WPE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O1VAWMONTH2GQ5MB1WYOSWR1IUCYU3C1ODPTYWIZQFNXWNTF
CLIENT_SECRET:LOX2OZZDHUQFVY1N2NEDLY0JS4QNIUWQYY0CPVYZJPFCWTKD


In [12]:
#!pip install geopy geocoder
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import geocoder # import geocoder

for index, row in df.iterrows():
    i=0
    postal_code = row['PostalCode']
    latitude = 0
    longitude = 0
    location = None
    while(location is None and i<2):
        #using geocoder
        #location = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        
        #using geolocator
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode('{}, Toronto, Ontario'.format(postal_code))
        
        i += 1
    if(location is not None):
        #geolocator
        latitude = location.latitude
        longitude = location.longitude
        #print(location.latitude,location.longitude)
        
        #google eocoder
        #latitude = lat_lng_coords[0]
        #longitude = lat_lng_coords[1]
        
    df.loc[index,'Latitude'] = latitude
    df.loc[index,'Longitude'] = longitude
df.head()

In [13]:
cvURI = 'https://cocl.us/Geospatial_data'
dfloc = pd.read_csv(cvURI)
dfloc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
for index, row in df.iterrows():
    match = None
    match = dfloc[dfloc['Postal Code']==row['PostalCode']]
    if(match is not None):
        longitude = match['Longitude'].values[0]
        latitude = match['Latitude'].values[0]
    else:        
        longitude = row['Longitude']
        latitude = row['Latitude']
    df.loc[index,'Latitude'] = latitude
    df.loc[index,'Longitude'] = longitude
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## 3. Neighborhood Exploration Analysis

#### Inspect our data give basic count summaries

In [15]:
neighborhoods = df

In [16]:
print('The dataframe has {} boroughs, {} Postal Codes, and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),len(neighborhoods['PostalCode'].unique()),
        len(neighborhoods['Neighbourhood'].unique()),
    )
)

The dataframe has 10 boroughs, 103 Postal Codes, and 99 neighborhoods.


In [17]:
#### How many unique neighbourhoods are there?

In [18]:
df.groupby(['Neighbourhood']).count().shape

(99, 4)

In [19]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ontario_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto, Canada with neighborhoods superimposed on top.

In [20]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now let's isolate the Neighbourhood of North York Borough for analysis

In [52]:
north_york = df[df['Borough']=='North York']
north_york

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073
13,M3C,North York,Don Mills,43.725900,-79.340923
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
34,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [53]:
north_york.loc[0,'Latitude']

43.7532586

In [55]:
address = 'North York, Toronto, Canada'

geolocator = Nominatim(user_agent="ontario_explorer")
location = geolocator.geocode(address)
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
neighborhood_name = 'North York'

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of North York are 43.7543263, -79.44911696639593.


Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [56]:
# type your answer here
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=O1VAWMONTH2GQ5MB1WYOSWR1IUCYU3C1ODPTYWIZQFNXWNTF&client_secret=LOX2OZZDHUQFVY1N2NEDLY0JS4QNIUWQYY0CPVYZJPFCWTKD&v=20180604&ll=43.7543263,-79.44911696639593&radius=500&limit=50'

Send the GET request and examine the resutls

In [60]:
import requests

results = requests.get(url).json()
#results

We reuse the get_category_type function from class lab

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507
2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,C-Care Health Services,Business Service,43.753644,-79.446977


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## 4 Explore Neighbourhoods in North York

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
# type your answer here
north_york_venues = getNearbyVenues(names=north_york['Neighbourhood'],
                                   latitudes=north_york['Latitude'],
                                   longitudes=north_york['Longitude']
                                  )

In [65]:
print(north_york_venues.shape)
north_york_venues.head()

(222, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [67]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,24,24,24,24,24,24
Downsview,14,14,14,14,14,14
"Fairview, Henry Farm, Oriole",50,50,50,50,50,50
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,3,3,3,3,3,3


Let's find out how many unique categories can be curated from all the returned venues

In [68]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 96 uniques categories.


### 5. Analyze Each Neighborhood

In [70]:
# one hot encoding
ny_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [71]:
ny_onehot.shape

(222, 97)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [72]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

ny_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.086957,...,0.043478,0.00,0.043478,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
1,Bayview Village,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.040000,0.00,0.04,0.00,0.04,0.00,0.000000,0.00
3,Don Mills,0.000000,0.000000,0.00,0.041667,0.000000,0.041667,0.000000,0.00,0.000000,...,0.041667,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
4,Downsview,0.000000,0.071429,0.00,0.000000,0.000000,0.000000,0.071429,0.00,0.071429,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.02,0.000000,0.000000,0.020000,0.000000,0.02,0.040000,...,0.000000,0.02,0.000000,0.02,0.00,0.02,0.04,0.02,0.000000,0.04
6,Glencairn,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
7,Hillcrest Village,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
8,Humber Summit,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00
9,"Humberlea, Emery",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00


Let's confirm the new size

In [73]:
ny_grouped.shape

(18, 97)

Let's print each neighborhood along with the top 5 most common venues

In [75]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.09
1                       Bank  0.09
2                       Park  0.04
3        Fried Chicken Joint  0.04
4  Middle Eastern Restaurant  0.04


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1  Italian Restaurant  0.08
2         Coffee Shop  0.08
3        Cupcake Shop  0.04
4             Butcher  0.04


----Don Mills----
                 venue  freq
0                  Gym  0.12
1           Beer Store  0.08
2          Coffee Shop  0.08
3           Restaurant  0.08
4  Japanese Restaurant  0.08


----Downsview----
                  venue  freq
0         Grocery Store  0.21
1                  Park  0.14
2          Home Se

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.


In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Sandwich Place,Intersection,Middle Eastern Restaurant,Ice Cream Shop,Park,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Diner,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Butcher,Greek Restaurant,Liquor Store,Café,Indian Restaurant,Comfort Food Restaurant,Pharmacy
3,Don Mills,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Discount Store,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Italian Restaurant
4,Downsview,Grocery Store,Park,Athletics & Sports,Home Service,Shopping Mall,Baseball Field,Gym / Fitness Center,Bank,Food Truck,Airport


### 6. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [100]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 4, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [102]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = north_york

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

ny_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,French Restaurant,Portuguese Restaurant,Hockey Arena,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Athletics & Sports,Construction & Landscaping,Cosmetics Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Discount Store,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Italian Restaurant
10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Pizza Place,Bakery,Pub,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


Finally, let's visualize the resulting clusters


In [108]:
# create map
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)
import math

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if(math.isnan(cluster)):
        cluster = 1
    else:
        cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 7. Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.


In [110]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Coffee Shop,French Restaurant,Portuguese Restaurant,Hockey Arena,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,North York,0.0,Clothing Store,Accessories Store,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Athletics & Sports,Construction & Landscaping,Cosmetics Shop
7,North York,0.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Discount Store,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Italian Restaurant
10,North York,0.0,Pizza Place,Bakery,Pub,Japanese Restaurant,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
13,North York,0.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Discount Store,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Italian Restaurant
27,North York,0.0,Golf Course,Mediterranean Restaurant,Pool,Fast Food Restaurant,Dog Run,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
28,North York,0.0,Coffee Shop,Bank,Gas Station,Sandwich Place,Intersection,Middle Eastern Restaurant,Ice Cream Shop,Park,Pharmacy,Pizza Place
33,North York,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Juice Bar,Bank,Women's Store,Toy / Game Store,American Restaurant,Pharmacy
34,North York,0.0,Furniture / Home Store,Caribbean Restaurant,Massage Studio,Bar,Coffee Shop,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
39,North York,0.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Diner,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop


# Thanks for taking your time to go through my work

_Folarin W.J. Martins_